In [1]:
import sys
sys.path.append("/Users/admin/Desktop/diploma")

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.optimize import minimize
from utilites import preprocess
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import RepeatedKFold
from scipy.optimize import minimize
import numpy.ma as mask
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample, check_random_state

#algos
from runc import RuncDualizer
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from models import BoostingElementaryPredicates, BoostingElementaryPredicatesv2
from bagging_trees import BaggingElementaryTrees

from time import time
from tqdm import tqdm
from math import sqrt
import math
import itertools
from sklearn.datasets import load_diabetes
from sklearn.datasets import make_regression
import numpy as np
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed
from own_forest import *
from utilites import *
from own_forest import *
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from memory_profiler import profile

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/Users/admin/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [ ]:
#pip install catboost or conda install -c conda-forge catboost (if using Anaconda or Miniconda)

In [2]:
import gc

gc.collect()

0

## Вспомогательные функции 

In [3]:
def hp_tuning(X_train, X_test, y_train, y_test, model, params, dataset_name, model_name):
    scoring_fnc = make_scorer(rmse, greater_is_better=False)
    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring=scoring_fnc, cv=3)
    grid_search = grid_search.fit(X_train, y_train)

    print(f"Best hyperparameters for {model_name} on {dataset_name}: {grid_search.best_params_}")
    best_model = grid_search.best_estimator_

    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)

    train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
    test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)

    print(f"\n--Train RMSE for {model_name} on {dataset_name}--\n{train_rmse}")
    print(f"\n--Test RMSE for {model_name} on {dataset_name}--\n{test_rmse}\n")

    return best_model, train_rmse, test_rmse

In [4]:
def visualize_errors(X_train, X_test, y_train, y_test, models, params):
    """
    Отображение изменения качества моделей при изменении гиперпараметров.
    
    Параметры:
    - X_train: обучающий набор признаков
    - X_test: тестовый набор признаков
    - y_train: обучающий набор целевой переменной
    - y_test: тестовый набор целевой переменной
    - models: список кортежей с моделями и их названиями
    - lb_params: словарь с диапазонами гиперпараметров
    """
    for model_name, model_class in models:
        test_results = []
        for num_iter in params["num_iter"]:
            model = model_class(num_iter=num_iter, **params['model_specific_params'])
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            test_results.append(rmse)
            print(f"{model_name} | Number of iters: {num_iter}, Test RMSE: {rmse}")

        plt.plot(params["num_iter"], test_results, label=model_name)

    plt.title("Test MSE across different num_iter")
    plt.legend()
    plt.xlabel("num_iter")
    plt.ylabel("RMSE")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [5]:
def visualize_scores(models, dataset, n_splits=10):
    """
    Визуализация боксплотов качества r2-score для различных моделей.
    
    Параметры:
    - models: список моделей с их названиями для сравнения
    - dataset: исходный датасет
    - n_splits: количество независимых запусков и разбиений
    """
    
    scores = {model_name: [] for model_name, _ in models}

    for _ in range(n_splits):
        # Предположим, что функция preprocess уже определена
        X_train, X_test, y_train, y_test = preprocess(dataset, 'TARGET_VARIABLE')
        
        for model_name, model in models:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            scores[model_name].append(r2)

    # Визуализация боксплотов
    scores_df = pd.DataFrame(scores)
    sns.boxplot(data=scores_df)
    plt.ylabel('R2 Score')
    plt.title('Comparison of R2 Scores across Models')
    plt.tight_layout()
    plt.show()

# Эксперименты на реальных данных 


## Датасет Air Quality

In [ ]:
#pip install ucimlrepo

In [6]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 
  
# metadata 
print(air_quality.metadata) 


{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 'S. D. Vito, E. Massera, M. P

In [7]:
air_quality.variables

,name,role,type,demographic,description,units,missing_values
0,Date,Feature,Date,None,None,None,no
1,Time,Feature,Categorical,None,None,None,no
2,CO(GT),Feature,Integer,None,True hourly averaged concentration CO in mg/m^...,mg/m^3,no
3,PT08.S1(CO),Feature,Categorical,None,hourly averaged sensor response (nominally CO...,None,no
4,NMHC(GT),Feature,Integer,None,True hourly averaged overall Non Metanic Hydro...,microg/m^3,no
5,C6H6(GT),Feature,Continuous,None,True hourly averaged Benzene concentration in...,microg/m^3,no
6,PT08.S2(NMHC),Feature,Categorical,None,hourly averaged sensor response (nominally NMH...,None,no
7,NOx(GT),Feature,Integer,None,True hourly averaged NOx concentration in ppb...,ppb,no
8,PT08.S3(NOx),Feature,Categorical,None,hourly averaged sensor response (nominally NOx...,None,no
9,NO2(GT),Feature,Integer,None,True hourly averaged NO2 concentration in micr...,microg/m^3,no


In [8]:
url = 'https://archive.ics.uci.edu/static/public/360/data.csv'
column_names = ['Time', 'CO(GT)','PT08.S1(CO)', 'NMHC(GT)','C6H6(GT)','PT08.S2(NMHC)',
                'NOx(GT)','PT08.S3(NOx)','NO2(GT)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']

dataset = pd.read_csv(url, skiprows=1, names=column_names,
                          na_values='?', comment='\t',
                          sep=',', skipinitialspace=True)


In [9]:
dataset.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [10]:
# Преобразование столбца Time из строкового формата в количество минут
dataset['Time'] = dataset['Time'].apply(lambda x: sum(int(a) * 60 for a in reversed(x.split(':'))))

In [11]:
X_train, y_train, X_val, y_val, X_test, y_test = preprocess(dataset, 'RH')

Будем использовать в качестве сравнения с моделью 3 различных реализации градиентного бустинга: LightGBM, Catboost и GBRegressor (from sklearn).

In [12]:
dataset_name = 'Air Quality'

lb_params =  {
    "num_iter" : np.linspace(1, 100, 20).astype(int),
    "m" : np.linspace(10, 20, 10).astype(int),
    "max_cov": [500]
}

lgbm_params = {
    'num_leaves': [x for x in range(2, 32, 8)], 
    'max_depth': [x for x in range(3, 7)],
    'learning_rate': [0.1]
}

cb_params = {
    'depth': [x for x in range(3, 10)], 
    'learning_rate': [0.1],
    'grow_policy': ['SymmetricTree', 'Depthwise'], 
    'score_function': ['Cosine', 'L2']
}

gb_params = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [3, 5, 7]
}

models = [
    ("BoostingElementaryPredicates", BoostingElementaryPredicatesv2(), lb_params),
    ("LightGBM", LGBMRegressor(), lgbm_params),
    ("CatBoost", CatBoostRegressor(), cb_params),
    ("GBRegressor", GradientBoostingRegressor(), gb_params)
]

results = []

for model_name, model, lb_params in models:
    _, train_rmse, test_rmse = hp_tuning(X_train, X_test, y_train, y_test, model, lb_params, dataset_name, model_name)
    results.append((dataset_name, model_name, train_rmse, test_rmse))

# Представление результатов
results_df = pd.DataFrame(results, columns=['Dataset', 'Model', 'Train RMSE', 'Test RMSE'])
print(results_df)

/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kw

In [ ]:
results_df

In [ ]:
results_df

,Dataset,Model,Train RMSE,Test RMSE
0,Air Quality,BoostingElementaryPredicates,17.983800,17.733886
1,Air Quality,LightGBM,0.684971,0.835571
2,Air Quality,CatBoost,0.159893,0.411092
3,Air Quality,GBRegressor,0.332681,0.621433


## Датасет NeorChem Prediction

Признаки слитка


        
      NPLAV - номер сертификата ститка. По этому идентификатору можно связать, какие детали были сделаны из одного слитка.

        
      MASS - масса слитка

        
      DIAMETER - диаметр слитка

        
      MASS_STRUG, N_MASS_STRUG - какие-то характеристики слитка (не знаю какие). Надо проверить, что они фиксированы для одинаковых NPLAV

        
      HIM_NAME_STR - список названий хим.элементов в слитке. Например, Al1 - аллюминий сверху, Al2 - аллюминий снизу.

        
      EL_ZN_STR - список значений хим. состава слитка, соответсвующий списку HIM_NAME_STR


Признаки изделия


        
      SOOTV_ID - ?????

        
      MFG_ORDER_ID - некий идентификатор заказа

        
      N_OBR - номер тестируемого образца

        
      NC_ZAK - номер заказа (по-моему, везде одинаковый)

        
      POS_KRAT - позиция в крате. Слиток распиливается на краты. Из крат вырезают заготовки. Каждой заготовке прискаивается номер

        
      KRATA - номер краты в слитке (оказалось, что форма зависимости от KRATA наших таргетов достаточно интересная - надо перепроверить)


Целевые переменные (мех. свойства детали)


        
      PRED_TEK_USL -пределел текучести условный, ksi

        
      VR_SOPR - временное сопротивление, ksi

        
      OTN_UDL - относительное удлинение, %

        
      OTN_SUZ - относительное сужение, %

In [14]:
# cd /Users/admin/Desktop/diploma

In [15]:
data = pd.read_csv("neor_him_meh.csv")
data

,NPLAV,MASS,DIAMETER,MASS_STRUG,N_MASS_STRUG,HIM_NAME_STR,EL_ZN_STR,PRED_TEK_USL,VR_SOPR,OTN_UDL,OTN_SUZ,SOOTV_ID,MFG_ORDER_ID,N_OBR,NC_ZAK,POS_KRAT,KRATA
0,0-08-03401,2184.0,650.0,0.0,908.2,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.08;6.19;0.20;0.20;0.005;0.005;0.024;0.020;0....,908.0,1007.0,18.3,41.9,1,НЕ ПЛАНИРУЕТСЯ В LPP,VS5953 (NO.5) FL 1,4,1,1
1,0-08-03401,2184.0,650.0,0.0,908.2,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.08;6.19;0.20;0.20;0.005;0.005;0.024;0.020;0....,897.0,988.0,17.9,41.0,1,НЕ ПЛАНИРУЕТСЯ В LPP,VS5953 FL 2,4,1,1
2,0-08-03401,2184.0,650.0,0.0,908.2,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.08;6.19;0.20;0.20;0.005;0.005;0.024;0.020;0....,884.0,978.0,17.5,41.8,1,НЕ ПЛАНИРУЕТСЯ В LPP,VS5954 FL 2,4,2,1
3,0-08-03401,2184.0,650.0,0.0,908.2,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.08;6.19;0.20;0.20;0.005;0.005;0.024;0.020;0....,891.0,990.0,17.5,40.6,1,НЕ ПЛАНИРУЕТСЯ В LPP,VS5955 FL 2,4,3,1
4,0-08-03401,2184.0,650.0,0.0,908.2,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.08;6.19;0.20;0.20;0.005;0.005;0.024;0.020;0....,885.0,984.0,17.9,41.8,1,НЕ ПЛАНИРУЕТСЯ В LPP,VS5956 FL 2,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4831,8-47-01272,4190.0,744.0,0.0,1788.8,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.48;6.52;0.21;0.18;0.0005;0.0007;0.013;0.013;...,887.0,980.0,17.9,44.6,1,04-0736882-19,VS9185 FL 2,4,5,6
4832,8-47-01272,4190.0,744.0,0.0,1788.8,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.48;6.52;0.21;0.18;0.0005;0.0007;0.013;0.013;...,899.0,994.0,17.9,42.5,1,04-0736882-19,VS9186 FL 2,4,6,6
4833,8-47-01272,4190.0,744.0,0.0,1788.8,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.48;6.52;0.21;0.18;0.0005;0.0007;0.013;0.013;...,909.0,1004.0,15.8,38.3,1,04-0736882-19,VS9187 FL 2,4,7,6
4834,8-47-01272,4190.0,744.0,0.0,1788.8,Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;...,6.48;6.52;0.21;0.18;0.0005;0.0007;0.013;0.013;...,894.0,991.0,16.7,39.7,1,04-0736882-19,VS9188 FL 2,4,8,6


In [16]:
# предобработка данных
unique_him_names = data['HIM_NAME_STR'].unique()

# Разбиение столбца EL_ZN_STR на столбцы с именами из HIM_NAME_STR
for i, him_name in enumerate(unique_him_names):
    data[f"HIM_NAME_{i+1}"] = data['EL_ZN_STR'].apply(lambda x: x.split()[unique_him_names.tolist().index(him_name)] if len(x.split()) > unique_him_names.tolist().index(him_name) else None)
unique_him_names

array(['Al1;Al2;Fe1;Fe2;H1;H2;N1;N2;O1;O2;C1;C2;V1;V2;Y1;Y2;ПК1;ПК2;ПС1;ПС2'],
      dtype=object)

In [17]:
data.dtypes

NPLAV            object
MASS            float64
DIAMETER        float64
MASS_STRUG      float64
N_MASS_STRUG    float64
HIM_NAME_STR     object
EL_ZN_STR        object
PRED_TEK_USL    float64
VR_SOPR         float64
OTN_UDL         float64
OTN_SUZ         float64
SOOTV_ID          int64
MFG_ORDER_ID     object
N_OBR            object
NC_ZAK            int64
POS_KRAT          int64
KRATA            object
HIM_NAME_1       object
dtype: object

In [18]:
X_train, X_test, y_train, y_test = preprocess(data, 'VR_SOPR')

AttributeError: 'DataFrame' object has no attribute 'data'

# Случайные данные 

In [6]:
from time import time

In [7]:
def simulate_training_time(sizes_m=np.arange(10, 21), n_features_list=[10, 20, 40, 60], n_samples=1000, num_iter=20):
    times = {n_features: [] for n_features in n_features_list}

    for n_features in n_features_list:
        print("Processing for n_features: ", n_features)
        for m in sizes_m:
            X, y = make_regression(n_samples=n_samples, n_features=n_features, noise=0.1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

            start_time = time()
            model = BoostingElementaryPredicates(num_iter=num_iter, m=m, max_cov=500)
            model.fit(X_train, y_train)
            times[n_features].append(time() - start_time)

    return sizes_m, times

def plot_times(sizes_m, times):
    plt.figure(figsize=(14, 8))
    n_subplots = len(times.keys())
    for i, (n_features, time) in enumerate(times.items(), 1):
        plt.subplot(n_subplots//2, 2, i)
        plt.plot(sizes_m, time, label=f'Features: {n_features}', marker='o')
        plt.title(f'Training Time for {n_features} Features')
        plt.xlabel('Number of Batches (m)')
        plt.ylabel('Time (s)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
    plt.show()

In [8]:
n_samples, boosting_times, bagging_times = simulate_training_time()
plot_times(n_samples, boosting_times, bagging_times)

Processing for n_features:  10
Processing for n_features:  20
Processing for n_features:  40


: 

: 

: 

In [ ]:
# datasets = [
#     ("Air Quality", "path_to_air_quality_dataset"), 
#     ("Russian Housing", "path_to_russian_housing_dataset"), 
#     ("House Sales", "path_to_house_sales_dataset")
# ]

# models = [
#     ("BoostingElementaryPredicates", BoostingElementaryPredicates(), lb_params_1),  # предположим, что lb_params_1 уже определены
#     ("LightGBM", LGBMRegressor(), lb_params_2),
#     ("CatBoost", CatBoostRegressor(), lb_params_3)
# ]

# results = []


# for dataset_name, dataset_path in datasets:
#     dataset = load_dataset(dataset_path)  # Предположение о функции загрузки
#     X_train, X_test, y_train, y_test = preprocess(dataset, 'TARGET_VARIABLE')  # предположим, что TARGET_VARIABLE уже определено

#     for model_name, model, lb_params in models:
#         _, train_rmse, test_rmse = tuning_and_visualization(X_train, X_test, y_train, y_test, model, lb_params, dataset_name, model_name)
#         results.append((dataset_name, model_name, train_rmse, test_rmse))


# results_df = pd.DataFrame(results, columns=['Dataset', 'Model', 'Train RMSE', 'Test RMSE'])
# print(results_df)